In [5]:
import pandas as pd
from math import *
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from random import sample
import utils
from utils import shuffling
from utils import normalization
from utils import traintestsplit
from utils import feature_extract
from utils import feature_extract
from utils import initializer
from utils import encode_label
from utils import maxminnorm
import os


In [6]:
dataset="bonn"

In [7]:
if dataset=="bonn":

        epilepsy_data = pd.read_csv("bonn_epilepsy.csv", sep =",")
        epilepsy_data.drop("Unnamed",axis=1,inplace=True)
        epilepsy_data.head()
        epilepsy_data.y = epilepsy_data.y==1
        epilepsy_data.y = epilepsy_data.y.astype(int)

        epilepsy_data= epilepsy_data[epilepsy_data.isnull().any(axis=1)==False]

        label = epilepsy_data["y"].astype("category").to_numpy()
        label2 = epilepsy_data["y"]
      
        epilepsy_data.drop("y",axis=1,inplace=True)
        epilepsy_data= feature_extract(epilepsy_data)
        epilepsy_data= epilepsy_data.iloc[:,-13:]
        #normalize the data
        normalized = normalization(epilepsy_data, label2)
elif dataset=="beirute":
    epilepsy_data = pd.read_csv("beirut_epilepsy.csv", sep =",")
    epilepsy_data.drop("Unnamed",axis=1,inplace=True)

    epilepsy_data = epilepsy_data.loc[(epilepsy_data["y"] == 0) | (epilepsy_data["y"] == 1)].reset_index()
    epilepsy_data.drop("index",axis=1,inplace=True)

    epilepsy_data.y = epilepsy_data.y.astype(int)
    epilepsy_data= epilepsy_data[epilepsy_data.isnull().any(axis=1)==False]

    label = epilepsy_data["y"].astype("category").to_numpy()
    label2 = epilepsy_data["y"]
    epilepsy_data.drop("y",axis=1,inplace=True)

    epilepsy_data = feature_extract(epilepsy_data)

    epilepsy_data= epilepsy_data.iloc[:,-5:]
    #normalize the data
    normalized = maxminnorm(epilepsy_data, label2)


In [8]:
#shuffle the data
shuffled = shuffling(normalized)
#split the data into train and test sets
X_train, X_test, Y_train, Y_test = traintestsplit(shuffled,0.15)

In [9]:
X_train.shape

(9775, 13)

In [10]:
np_train_data= X_train
np_test_data= X_test

np_label_train=Y_train
np_label_test=Y_test

In [11]:
def confusion_matrix_plot(true_label,predictions):
    

    class_labels = np.unique(true_label,return_counts=False)
    
    cm= confusion_matrix(true_label,predictions) 
    cp=ConfusionMatrixDisplay(cm,display_labels=class_labels)
    cp.plot()
    plt.title("Confusion Matrix")
    plt.show()

In [12]:
class Layer:
    
    def __init__(self,currNeurons, activFunction,name):
        self.currNeurons=currNeurons
        self.activFunction=activFunction
        self.name=name
        
        self.delta=None
        self.error=None
        self.cacheActiv=None
        self.weights=None
    
    def layer_activation(self,net):
        if self.activFunction=="relu":
            return np.maximum(0,net)
        elif self.activFunction=="sigmoid":
            return 1/(1+np.exp(-net))
        elif self.activFunction=="tanh":
            return np.tanh(net)
        elif self.activFunction=="softmax":
            return np.exp(net)/np.sum(np.exp(net),axis=0)
    
    def layer_derivative(self,activ):
        
        if self.activFunction=="relu":
            return 1*(activ>0)
        elif self.activFunction=="sigmoid":
            return activ*(1-activ)
        elif self.activFunction=="tanh":
            return 1-activ**2
        elif self.activFunction=="softmax":
               return activ*(1-activ)
            
      
            
        

        
        

In [13]:
class Sequential:
    
    def __init__(self,inputDim):
        self.layers=[]
        self.inputDim=inputDim
        
        self.loss=None
        self.lr=0.01
        
        
    
    def layer_addition(self,layer):
        self.layers.append(layer)
    
    def compile_(self,lossFn,learning_rate,weight_initializer):
        
        if lossFn=="mse":
            self.loss="mse"
        elif lossFn=="cross_entropy":
            self.loss="cross_entropy"
        
        self.lr=learning_rate
        self.weight_initializer= weight_initializer
           
        for i in range(len(self.layers)):
          
            lyr=self.layers[i]
            if i==0:
                lyr.weights= initializer(init_choice=self.weight_initializer,shape=(self.layers[0].currNeurons,self.inputDim+1))
            else:
                lyr.weights= initializer(init_choice=self.weight_initializer,shape=(self.layers[i].currNeurons,self.layers[i-1].currNeurons+1))
        
    def forward_propagation(self,inp):
        
        output=inp
     
        for lyr in self.layers:
           
            num_samples=output.shape[1]
            output=np.r_[output,[np.ones(num_samples)*1]]
            output=lyr.layer_activation(np.matmul(lyr.weights,output))
           
            lyr.cacheActiv=output
         
        
        return output
    
    def predict(self, data):
        out = self.forward_propagation(data)
        return np.argmax(out, axis=0) 
    
    
    
    
    def backward_propagation(self,inp,out):
        batch_size= inp.shape[1]
        network_output=self.forward_propagation(inp)
        
        for ly in reversed(range(len(self.layers))):

                layer= self.layers[ly]
                if layer == self.layers[len(self.layers)-1]:
                    
                    if self.loss=="cross_entropy" and layer.activFunction=="softmax":
                        layer.delta=network_output-out
                     
                    
                    elif self.loss=="mse":
                        
                        layer.error = network_output-out
                        errorDerivative = layer.layer_derivative(layer.cacheActiv)
                        layer.delta=layer.error*errorDerivative
                    else:
                        print("Sorry, I do not want to perform this operation.")
                
                else:
                        layerNext = self.layers[ly+1]
                        layerNextWeights= layerNext.weights[:,:-1]
                        layer.error = np.matmul(layerNextWeights.T,layerNext.delta)
                        errorDerivative= layer.layer_derivative(layer.cacheActiv)
                        layer.delta=errorDerivative*layer.error
                        
        
        for ly in range(len(self.layers)):
          
            if ly==0:
            
                activationInput=np.r_[inp,[np.ones(batch_size)*1]]
          
            else:
                activationInput=np.r_[self.layers[ly-1].cacheActiv,[np.ones(batch_size)*1]]
            
            
            dW= np.matmul(self.layers[ly].delta,activationInput.T)/batch_size
            
            self.layers[ly].weights=self.layers[ly].weights-self.lr*dW
            
            
    def loss_calculation(self,data,label):
        
            val_out = self.forward_propagation(data.T)
            val_label=label
            if(self.loss == 'cross_entropy'):
                err = - np.sum(np.log(val_out)*encode_label(val_label).T)/val_out.shape[1]
            elif(self.loss == 'mse'):
                err = np.sum((encode_label(val_label).T - val_out)**2)/val_out.shape[1]
            return err
                    
        
        
        
    def fit(self,data,label,val_ratio,epoch,batch_size):
        trainError= []
        trainAcc=[]
        valError=[]
        valAcc=[]
        
        
        for e in range(epoch):
        
        
           
            print('=====================================================\nEpoch', e+1)
            indx= np.random.permutation(data.shape[0])
            data_temp=data[indx]
            label_temp=label[indx]
            
            val_range=int(data.shape[0]*val_ratio)
           
            val_data= data[:val_range]
          
            val_label=label[:val_range]
            
            data_temp=data_temp[val_range:]
            label_temp=label_temp[val_range:]
            
            iterations = int(np.floor(len(data)/batch_size))
            
            for it in range(iterations):
                
                data_batch=data_temp[it*batch_size:it*batch_size+batch_size]
                label_batch=label_temp[it*batch_size:it*batch_size+batch_size]
                if label_batch.shape[0]!=0:
                    label_batch = encode_label(label_batch)            
                    self.backward_propagation(data_batch.T, label_batch.T)
                
            err_tr= self.loss_calculation(data_temp,label_temp)
            err_val=self.loss_calculation(val_data,val_label)
            valError.append(err_val)
            trainError.append(err_tr)
            if self.loss=="cross_entropy":
                print("CE loss in training: ", err_tr)
                print("CE loss in training: ", err_val)
            else:
                print("MSE loss in training: ", err_tr)
                print("MSE loss in training: ", err_val)
                

           
                        
            pred_train = self.predict(data.T)
        
            #trAcc = np.sum(trPred.reshape((trPred.shape[0],1)) == out)/trPred.shape[0]*100
            acc_train=np.sum(pred_train==label)/pred_train.shape[0]
            print('Training Accuracy: ', acc_train*100)
            trainAcc.append(acc_train*100)
            
            pred_val = self.predict(val_data.T)
            acc_val= np.sum(pred_val==val_label)/pred_val.shape[0]
            print('Validation Accuracy: ', acc_val*100)
            valAcc.append(acc_val*100)
            
        
        return trainAcc, valAcc, valError,trainError
        
        
        

# Here, create your models

In [16]:
mlp= Sequential(X_train.shape[1]) 
mlp.layer_addition(Layer(50,"tanh","layer1"))
mlp.layer_addition(Layer(20,"tanh","layer2"))
mlp.layer_addition(Layer(10,"tanh","layer3"))
mlp.layer_addition(Layer(2,"softmax","layer4"))
mlp.compile_("cross_entropy",0.5,"normal")

In [18]:
trainAcc, valAcc, valError,trainError = mlp.fit(np_train_data,np_label_train,0.2,5,50)

Epoch 1
CE loss in training:  0.08748847737629761
CE loss in training:  0.08293063051606027
Training Accuracy:  96.51150895140665
Validation Accuracy:  96.98209718670077
Epoch 2
CE loss in training:  0.08603435122423124
CE loss in training:  0.07952862290456779
Training Accuracy:  96.46035805626599
Validation Accuracy:  96.82864450127877
Epoch 3
CE loss in training:  0.08612219371049314
CE loss in training:  0.0777338252391605
Training Accuracy:  96.65473145780051
Validation Accuracy:  97.03324808184142
Epoch 4
CE loss in training:  0.08648514661879485
CE loss in training:  0.07764951460591224
Training Accuracy:  96.66496163682864
Validation Accuracy:  97.2378516624041
Epoch 5
CE loss in training:  0.09151691325057969
CE loss in training:  0.07931874486334914
Training Accuracy:  96.27621483375958
Validation Accuracy:  96.62404092071611


In [19]:
testPred = mlp.predict(np_test_data.T)
testAcc = np.sum(testPred ==np_label_test)/testPred.shape[0]
print(testAcc*100)

96.05797101449275
